From /beagle3/roux/kyleghaby/kininhibs/main/

16 Dec 2025

In [1]:
import re
#import mdtraj
from shutil import copyfile
import os
import sys
from IPython.display import Markdown as md
import math
import numpy as np
import parmed as pmd #https://github.com/ParmEd/ParmEd
import nglview #https://github.com/nglviewer/nglview
#vmd (linux ver) also required
#AMBERTOOLS (cpptraj) required
#pdb_tools also required
#updated toppar: http://mackerell.umaryland.edu/charmm_ff.shtml#charmm

In [86]:
os.getcwd()

'/mnt/c/Users/Kyle/Desktop/Roux/kininhibs/main/TAK1_SM1/build'

In [3]:
os.mkdir('build')

In [4]:
os.chdir('build')

In [ ]:
#notes in README
#filled residues: 

## Setup

### 1. Download and process PDB

- keep xtal waters with the protein
- run the code below. 

In [88]:
PDBID='5J7S'
ligResname='6H3'

#fetch pdb and fasta, renumber residues to start with 1, and discard alternative coordinates
!./../../code/get_xtal.sh {PDBID}

!mkdir -p modeller
!mkdir -p scwrl

--2023-02-16 11:09:42--  https://www.rcsb.org/fasta/entry/5J7S
Resolving www.rcsb.org (www.rcsb.org)... 128.6.159.248
Connecting to www.rcsb.org (www.rcsb.org)|128.6.159.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 465 [text/x-fasta]
Saving to: ‘5J7S.fasta’

5J7S.fasta          100%[===================>]     465  --.-KB/s    in 0s      

2023-02-16 11:09:42 (43.5 MB/s) - ‘5J7S.fasta’ saved [465/465]

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 

### 2. Polish protein structure

- Fill in missing residues in xtal/{PDBID}_noaltloc.pdb using Modeller in Chimera
    1. Open PDB file, then show its Sequence (Chimera menu: Tools… Sequence… Sequence) or skip to step D if you have the full sequence file already
    2. Save the sequence as FASTA file (sequence window menu: File… Save As, choose file type: aligned FASTA)
    3. Edit the FASTA file to add the new sequence
    4. Quit from the first sequence window and open the new fasta file in Chimera.
    5. Associate it with the structure (sequence window menu: Structure… Associations, change from "none" to the new sequence)
    6. Use the Modeller interface (sequence window menu:  Structure… Modeller (loops/refinement)) to model "non-terminal missing structure" or "all missing structure". Make sure this also fills in missing side chains of all residues.            
        -  Note for terminal regions of kinases: 
           Cterm does not affect kinase activity so leave as is. N term might be important. The tryptophan (AKDAW) is needed bc wedge (W260 in Src (2OIQ)) and D258 is probably also important bc salt bridge. Most include AKDAW. Good to aim for including AKDAW.
    7. Save output as modeller/{PDBID}_noaltloc.pdb.
    8. While modeller is open, note the side chains that were incomplete. SCWRL is used to remodel the side chains for those residues.

- Renumber residues and make a new pdb with just prot/wat (code below)
    - not strictly necessary since modeller output will likely only have protein and water and will be renumbered, but it helps to rename the files so later steps can expect those files to exist.
- Model side chains not in xtal with SCWRL (code below)

In [92]:
#renumber residues to start with 1
!pdb_reres modeller/{PDBID}_noaltloc.pdb > modeller/{PDBID}_noaltloc_reres.pdb 

#remove lig (and other residues) from prot/wat (assuming pdb only contains prot,lig,wat)
!pdb_delresname -{ligResname} modeller/{PDBID}_noaltloc_reres.pdb  > modeller/prot_wat.pdb

#make a ligand-only pdb file in xtal
!pdb_selresname -{ligResname} xtal/{PDBID}_noaltloc.pdb  > xtal/{ligResname}_noaltloc.pdb

In [ ]:
# Generate SCWRL input

def find_incomplete_residues(pdb_file, amino_acids):
    '''
    Function to detect residues with incomplete side chains in PDB
    '''
    incomplete_residues = set()
    current_residue = None
    current_atoms = set()
    first_res=None
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                res_name = line[17:20].strip()
                res_seq = int(line[22:26].strip())
                atom_name = line[12:16].strip()
                if not first_res:
                    first_res=res_seq
                if current_residue != (res_name, res_seq):
                    if current_residue:
                        expected_atoms = set(amino_acids.get(current_residue[0], []))
                        if not expected_atoms.issubset(current_atoms):
                            incomplete_residues.add(current_residue[1])

                    current_residue = (res_name, res_seq)
                    current_atoms = set()

                current_atoms.add(atom_name)

    # Check last residue
    if current_residue:
        expected_atoms = set(amino_acids.get(current_residue[0], []))
        if not expected_atoms.issubset(current_atoms):
            incomplete_residues.add(current_residue[1])
 
    sorted_set=sorted(incomplete_residues)
    incomplete_residues={num - (first_res - 1) for num in sorted_set}
    print('Residues with incomplete side chains:',', '.join(f"{num} ({num - (first_res - 1)})" for num in sorted_set),'\n')
    return incomplete_residues


def align_and_capitalize(sequence_with_gaps, sequence_with_incomplete):
    """
    Aligns two sequences and returns the first sequence with residues capitalized based on the second sequence.
    """
    # Initialize pointers and output list
    ptr_gaps = 0
    ptr_incomplete = 0
    output = []
    
    # Loop through the sequence with gaps
    while ptr_gaps < len(sequence_with_gaps):
        residue_gaps = sequence_with_gaps[ptr_gaps]
        
        # Handle unmodeled residues (capitalized in sequence_with_gaps)
        if residue_gaps.isupper():
            output.append(residue_gaps)
            ptr_gaps += 1
            continue
        
        # Loop through the sequence with incomplete side chains
        while ptr_incomplete < len(sequence_with_incomplete):
            residue_incomplete = sequence_with_incomplete[ptr_incomplete]
            
            # Match found
            if residue_gaps.lower() == residue_incomplete.lower():
                # Capitalize if the residue in sequence_with_incomplete is capitalized
                if residue_incomplete.isupper():
                    output.append(residue_gaps.upper())
                else:
                    output.append(residue_gaps.lower())
                
                ptr_incomplete += 1
                break
            
            ptr_incomplete += 1
        
        ptr_gaps += 1
    
    return ''.join(output)

def extract_seq_and_residues(pdb_lines):
    '''
    Function to extract sequence and residue numbers from a PDB file
    '''
    seq = []
    residues = []
    for line in pdb_lines:
        if line.startswith("ATOM") and line[13:15].strip() == "CA":
            res_num = int(line[22:26].strip())
            amino_acid = line[17:20].strip()
            # Using standard amino acid codes (IUPAC)
            aa_dict = {'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
                       'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
                       'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
                       'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'}
            seq.append(aa_dict.get(amino_acid, 'X'))
            residues.append(res_num)
    return ''.join(seq), residues

def find_extensions_and_insertions(xtal_seq, xtal_residues, modeller_seq):
    # Create a dictionary to map xtal residues to their positions 
    xtal_map = {res: aa for res, aa in zip(xtal_residues, xtal_seq)}
    
    # Generate gap-filled sequence for xtal_seq
    gap_filled_xtal_seq = []
    for i in range(min(xtal_residues), max(xtal_residues) + 1):
        gap_filled_xtal_seq.append(xtal_map.get(i, '-'))

    # Shift modeller_seq and identify insertions
    max_overlap = 0
    best_shift = 0
    for shift in range(len(modeller_seq) - len(gap_filled_xtal_seq) + 1):
        overlap = 0
        for i in range(len(gap_filled_xtal_seq)):
            if gap_filled_xtal_seq[i] == modeller_seq[i + shift] or gap_filled_xtal_seq[i] == '-':
                overlap += 1
        if overlap > max_overlap:
            max_overlap = overlap
            best_shift = shift

    # Generate the output sequence 
    output_seq = []
    for i in range(len(modeller_seq)):
        if i < best_shift or i >= best_shift + len(gap_filled_xtal_seq):
            output_seq.append(modeller_seq[i].upper())
        else:
            if gap_filled_xtal_seq[i - best_shift] == '-':
                output_seq.append(modeller_seq[i].upper())
            else:
                output_seq.append(modeller_seq[i])

    return ''.join(output_seq)

# Define amino acid side-chain atoms 
amino_acids = {
    'ALA': ['CB'],
    'ARG': ['CB', 'CG', 'CD', 'NE', 'CZ', 'NH1', 'NH2'],
    'ASN': ['CB', 'CG', 'OD1', 'ND2'],
    'ASP': ['CB', 'CG', 'OD1', 'OD2'],
    'CYS': ['CB', 'SG'],
    'GLN': ['CB', 'CG', 'CD', 'OE1', 'NE2'],
    'GLU': ['CB', 'CG', 'CD', 'OE1', 'OE2'],
    'GLY': [],
    'HIS': ['CB', 'CG', 'ND1', 'CD2', 'CE1', 'NE2'],
    'ILE': ['CB', 'CG1', 'CG2', 'CD1'],
    'LEU': ['CB', 'CG', 'CD1', 'CD2'],
    'LYS': ['CB', 'CG', 'CD', 'CE', 'NZ'],
    'MET': ['CB', 'CG', 'SD', 'CE'],
    'PHE': ['CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
    'PRO': ['CB', 'CG', 'CD'],
    'SER': ['CB', 'OG'],
    'THR': ['CB', 'OG1', 'CG2'],
    'TRP': ['CB', 'CG', 'CD1', 'CD2', 'NE1', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2'],
    'TYR': ['CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'OH'],
    'VAL': ['CB', 'CG1', 'CG2']
}


# Load PDB and Fasta files
xtal_pdb = f'xtal/{PDBID}_noaltloc.pdb'
modeller_pdb = f'modeller/prot_wat.pdb'

# Extract sequences and residue numbers
xtal_pdb_lines = [line.strip() for line in open(xtal_pdb, 'r')]
modeller_pdb_lines = [line.strip() for line in open(modeller_pdb, 'r')]
xtal_seq, xtal_residues = extract_seq_and_residues(xtal_pdb_lines)
xtal_seq=xtal_seq.lower()
print(f"====XTAL({len(xtal_seq)})====",xtal_seq,'\n')
modeller_seq, modeller_residues = extract_seq_and_residues(modeller_pdb_lines)
modeller_seq=modeller_seq.lower()
print(f"====MODELLER({len(modeller_seq)})====",modeller_seq,'\n')

# Identify gaps and extensions
sequence_with_gaps = find_extensions_and_insertions(xtal_seq, xtal_residues, modeller_seq)
print(f"====GAPS({len(sequence_with_gaps)})====", sequence_with_gaps,'\n')

# Identify incomplete residues
incomplete_residues = find_incomplete_residues(xtal_pdb, amino_acids)
sequence_with_incomplete = ""
for i, aa in enumerate(xtal_seq):
    if i + 1 in incomplete_residues:
        sequence_with_incomplete += aa.upper()
    else:
        sequence_with_incomplete += aa.lower()
print(f"====INCOMPLETE SIDE CHAINS({len(sequence_with_incomplete)})====",sequence_with_incomplete,'\n')

# Generate final SCWRL sequence
final_scwrl_sequence = align_and_capitalize(sequence_with_gaps, sequence_with_incomplete)
print(f"====SCWRL({len(final_scwrl_sequence)})====",final_scwrl_sequence,'\n')

# Write final sequence to scwrl.seq
with open('scwrl/scwrl.seq', 'w') as f:
    f.write(final_scwrl_sequence)

print("Check ambiguous residues and edit scwrl/scwrl.seq to change capitalization if needed. Ambigious residues are those with adjacent repeats, leading to ambiguity about the exact location of the gap. This ambiguity can be resolved by checking with residue is linked in the PDB.")


====XTAL(267)==== cvkieevigagevcrgrlkqpgrrevfvaiktlkvgyterqrrdflseasimgqfdhpniirlegvvtksrpvmiltefmencaldsflrlndgqftviqlvgmlrgiaagmkylsemnyvhrdlaarnilvnsnlvckvsdfgleddpsdptytsslggkipirwtapeaiayrkftsasdvwsygivmwevmsygerpywdmsnqdvinaveqdyrlpppmdcptalhqlmldcwvrdrnlrpkfsqivntldklirnpaslkvi 

====MODELLER(274)==== cvkieevigagefgevcrgrlkqpgrrevfvaiktlkvgyterqrrdflseasimgqfdhpniirlegvvtksrpvmiltefmencaldsflrlndgqftviqlvgmlrgiaagmkylsemnyvhrdlaarnilvnsnlvckvsdfglsrfleddpsdptytsslggkipirwtapeaiayrkftsasdvwsygivmwevmsygerpywdmsnqdvinaveqdyrlpppmdcptalhqlmldcwvrdrnlrpkfsqivntldklirnpaslkvi 

====GAPS(274)==== cvkieevigaGEFgevcrgrlkqpgrrevfvaiktlkvgyterqrrdflseasimgqfdhpniirlegvvtksrpvmiltefmencaldsflrlndgqftviqlvgmlrgiaagmkylsemnyvhrdlaarnilvnsnlvckvsdfglSRFLeddpsdptytsslggkipirwtapeaiayrkftsasdvwsygivmwevmsygerpywdmsnqdvinaveqdyrlpppmdcptalhqlmldcwvrdrnlrpkfsqivntldklirnpaslkvi 

Residues with incomplete side chains: 636 (5), 637 (6), 651 (20), 657 (26), 658 (27), 659 (28), 668 (37), 669 (38), 67

In [98]:
# Run SCWRL command
!scwrl4 -i modeller/prot_wat.pdb -s scwrl/scwrl.seq -f xtal/{PDBID}_noaltloc.pdb -o scwrl/prot_wat.pdb


Using parameters from /mnt/c/FCCC/Scwrl4_linux//Scwrl4.ini

<Timing tag = "Loading_PDB_file"  duration = "0.016" />

Using rotamer library: /mnt/c/FCCC/Scwrl4_linux/bbDepRotLib.bin

Residue A   1  is being treated as N-terminal

Residue A 306  is being treated as C-terminal

<Timing tag = "Adding_residues"  duration = "0.016" />

<Timing tag = "Loading_FrameFile"  duration = "0.000" />

Sequence file has been loaded
---------------
"slhmidykeieveevvgrgAFgvvckakwrakdvaikQIESESERkafivelrqlsrvnhpnivklyGAClnpvclvmeyaeggslynvlhgaeplpyytaahamswclqcsqgvaylhsmqpkalihrdlkppnlllvaggtvlkicdfgTACDIQTHMTNNKGsaawmapevfegsnysekcdvfswgiilwevitrrkpfdeiggpafrimwavhngtrppliknlpkpieslmtrcwskdpsqrpsmeeivkimthlmryfpgadeplqypcqhslppgedgrvepyvdfaefyrlwsvdhg"
---------------

<Timing tag = "Building_Atoms_Sets"  duration = "0.000" />

<Timing tag = "Building_Flexible_Subrotamers"  duration = "0.000" />

<Timing tag = "Computing_Symmetry_Operators"  duration = "0.000" />

<Timing tag = "Updating_Self_Energies"

In [ ]:
# Add the xtal waters to scwrl output 

# Initialize counters
atom_count = 0
hetatm_count = 0
hoh_count = 0
wat_count = 0

# Open source PDB file for reading and destination PDB file for appending
with open('modeller/prot_wat.pdb', 'r') as src, open('scwrl/prot_wat.pdb', 'a') as dest:
    for line in src:
        # Check for ATOM or HETATM and water residues (HOH or WAT)
        if line.startswith('ATOM'):
            residue = line[17:20].strip()
            if residue in ('HOH', 'WAT'):
                atom_count += 1
                if residue == 'HOH':
                    hoh_count += 1
                else:
                    wat_count += 1
                dest.write(line)
        elif line.startswith('HETATM'):
            residue = line[17:20].strip()
            if residue in ('HOH', 'WAT'):
                hetatm_count += 1
                if residue == 'HOH':
                    hoh_count += 1
                else:
                    wat_count += 1
                dest.write(line)

# Logging (counters used for efficient single-pass tallying)
print(f"Total waters: {hoh_count + wat_count}")
print(f"ATOM: {atom_count}, HETATM: {hetatm_count}")
print(f"HOH: {hoh_count}, WAT: {wat_count}")


Total waters: 94
ATOM: 0, HETATM: 94
HOH: 94, WAT: 0


In [ ]:
# Renumber structure
def renumber_pdb(file_path):
    '''
    Function to renumber atoms and residues in a PDB file
    '''
    atom_counter = 1  # Initialize atom counter
    residue_counter = 0  # Initialize residue counter
    last_residue = None  # Track last residue ID
    
    # Read original lines from PDB
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Open PDB file for writing
    with open(file_path, 'w') as f:
        for line in lines:
            # Check for ATOM or HETATM lines
            if line.startswith(('ATOM', 'HETATM')):
                # Update atom number (columns 7-11)
                line = line[:6] + f"{atom_counter:5d}" + line[11:]
                
                # Check if the residue ID has changed (columns 23-26)
                current_residue = line[22:26].strip()
                if current_residue != last_residue:
                    residue_counter += 1
                last_residue = current_residue
                
                # Update residue number (columns 23-26)
                line = line[:22] + f"{residue_counter:4d}" + line[26:]
                
                # Increment atom counter
                atom_counter += 1
                
            # Write the modified or unmodified line
            f.write(line)

# File path to the PDB file that needs to be renumbered
file_path = 'scwrl/prot_wat.pdb'

# Renumber the PDB file
renumber_pdb(file_path)

In [114]:
# check pdbs


check='''
set filelist {"xtal/PDBID_noaltloc.pdb" "modeller/prot_wat.pdb" "scwrl/prot_wat.pdb"}

foreach i $filelist {
  echo " "
  echo "Loading $i"
  mol new "$i" type {pdb} waitfor all
  echo " "
}
'''

check=check.replace("PDBID",PDBID)

with open("scwrl/check.tcl", "w") as check_fh:
    check_fh.write(check)
!vmd -e scwrl/check.tcl

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 9GB (55%)
Info) No CUDA accelerator devices available.
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
Info) OpenGL renderer: NVIDIA GeForce GTX 1070 Ti/PCIe/SSE2
Info)   Features: STENCIL MSAA(4) MDE MTX NPOT PP PS 
Info)   GLSL rendering mode is NOT available.
Info)   Textures: 2-D (32768x32768), 3-D (16384x16384x16384), Multitexture (4)
Info) Dynamically loa

### 3. Use CHARMM-GUI solution builder steps 1-3 to make a system comprising water, ions, and protein. 

- Change xtal water segid to WATA so it doesnt clash with SOLV segment used by GUI
- Fix resnames
    - If aspartic acids are separated from solvent, local pH causes it to be protonated. 
    - Fix histidine resnames. It may not matter but its correct. HIS to HSD/HSE.
        - eg. HIS384 in 2OIQ should be HSE, and the atom name was changed manually. HSE and HSD says which nitrogen is protonated. Programs may default to HSD. Rename to HSD or HSE with the GUI mutation option. but GUI might defualt to HSD anyway so you prob only need to mutate to HSE.
- Add disulfides if needed. 
- Edge distance of 12 fit to protein size (rect)
- 0.15 M KCl (intracellular)
- Note the PBC used: 

In [6]:
#dimensions of periodic boundaries established in the precursor steps with charmm-gui. 
#Format string as comma-delineated string of dimension lengths to be used in chamber box option (eg pbc="89,89,89")
pbc="93,93,93"

# HIS ___ were mutated into HSE

### 4. Align:

- Old reference pdb with ligand to new step3 pdb (step3 PDB contains coords to fit with PBC).
    - pymol
- Extract aligned ligand coordinates to replace old ligand coordinates. (code below)
    - pdb_tools

In [ ]:
!pdb_selresname -{lig_resname} {PDBID}_aligned2charmm.pdb | pdb_reres > {lig_resname}_aligned2charmm.pdb

### 5. Make ligand PDBs and mol2's.

   - Using Pymol:
        - Ligand with attached residue with acetylated N-term and N-methylamidated C-term (patch coordinates)
            1. Use pymol builder to get linked state, add caps, and change bond orders. Make sure the linking atom has its hydrogens in the right pose relative to the true link so that the linking carbon is tetrahedral. 
            2. Save as pdb. When saving, enable the following:
                - Write multiple bonds as duplicate CONECT records
                - Write CONECT records for all bonds
                - Write segment identifier (segi) column
            3. Put ligand residue first so atomnames in ligand arent replaced by CGenFF. Rename attached residues atomnames with the following nomenclature (unless they clash with ligand atomnames! There cant be any duplicate atomnames!) Move the atom name one space to the left in the PDB if it becomes 4 digits (eg HM10 in lysine should be moved one space toward "ATOM" rather than having the "0" go into the first resname column):
                - N-term cap residue:
                    - CA  -> CN1
                    - C -> CN2
                    - O -> ON1
                    - H0# -> HN#
                - Middle residue:
                    - N -> NM1
                    - H0# -> HM#
                - C-term cap residue:
                    - N -> NC1
                    - CA -> CC1
                    - H0# -> HC#
            4. Rename ligand residue, change ligand chain to B, rename H's, and renumber atoms and residues (code below)
            5. Load back into pymol and save as mol2.
        - Ligand by itself (ligand coordinates)
            1. From a duplicate of the ligand_res pdb, use pymol builder to remove prot atoms and change linked state back to unlinked state while affecting pose as little as possible. Note: Any H's affected by pymol builder will have to be renamed back to IUPAC standard.
            2. Save as pdb. When saving, enable the following:
                - Write multiple bonds as duplicate CONECT records
                - Write CONECT records for all bonds
                - Write segment identifier (segi) column
            3. Rename added atoms (hydrogen for example) if needed and renumber atoms and residues if needed (code below).
            4. Load back into pymol and save as mol2.
            
   - Check: 
            - Bond orders and connectivity in ligand mol2's are correct 

In [7]:
#function for renaming PDB H's 
#has an atomnumber limit of 99999 

def renameH(pdbfile):

    hatm_l=list()
    tempfile=pdbfile+".temp"
    tempfh = open(tempfile,"w+")
    with open(pdbfile, 'r') as pdbfh :
        lines = pdbfh.readlines()
    print('Connecting Atom\tOld Name\tNew Name')
    for hatmline in lines:
        if hatmline.startswith('ATOM'):
            oldname=hatmline[11:16].strip()
            if oldname.startswith('H'):
                hatmnbr=hatmline[9:11].strip()
                for conline in lines:
                    if conline.startswith('CONECT'):
                        if hatmnbr == conline[6:11].strip():
                            catmnbr = conline[11:16].strip()
                            for catmline in lines:
                                if catmline.startswith('ATOM'):
                                    if catmnbr == catmline[9:11].strip():
                                        catmname=catmline[11:16].strip()
                                        catmname_suffix=catmname[1:]
                                        n=str(1)
                                        newname='H'+catmname_suffix+n
                                        while newname in hatm_l:
                                            n=str(int(n)+1)
                                            newname='H'+catmname_suffix+n
                                        hatm_l.append(newname)
                                        hatmline = hatmline[:11]+newname.rjust(5)+hatmline[16:]
                                        
                                        print('{0}\t\t{1}\t\t{2}'.format(catmname,oldname, newname))
        tempfh.write(hatmline)    
    tempfh.close()
    !mv $tempfile $pdbfile
    return


In [8]:

def alter_pdb(pdbfile,oldresname,newresname,oldchain):
    print('\nAltering {}'.format(pdbfile))
    !cp $pdbfile "$pdbfile".bu
    print("Made backup of "+pdbfile)

    #remove TER lines
    fh = open(pdbfile, "r")
    lines = fh.readlines()
    fh.close()

    fh = open(pdbfile, "w")
    for line in lines:
        if line[0:3] != "TER":
            fh.write(line)
    fh.close()
    print("Removed TER lines")
    
    #replace HETATM with ATOM
     # Read in the file
    with open(pdbfile, 'r') as pdbfh :
        pdbfiledata = pdbfh.read()
     # Replace the target string
    pdbfiledata = pdbfiledata.replace('HETATM', 'ATOM  ')
     # Write the file out again
    with open(pdbfile, 'w') as pdbfh:
        pdbfh.write(pdbfiledata)
    print("Changed HETATM to ATOM")
    
    #renumber atoms and residues 
    !pdb_reres $pdbfile | pdb_reatom > "temp.pdb"
    !mv "temp.pdb" $pdbfile
    print("Renumbered atoms and residues")
    
    #change residue name and chain ID of ligand PDB
    !pdb_rplresname -"$oldresname":"$newresname" $pdbfile | pdb_rplchain -"$oldchain":B > "temp.pdb"
    !mv "temp.pdb" $pdbfile
    print("Changed resname from {} to {} and changed chain from {} to B".format(oldresname,newresname,oldchain))
    
    renameH(pdbfile)
    print('Renamed hydrogens\n')
    return



In [12]:
#5.LINKED.D
#manually cut and paste the ligand residue to the top of the PDB

#MANUALLY CHANGE ARGUMENTS IN THIS COMMAND (FILE,OLDRESNAME,NEWRESNAME,OLDCHAIN)
alter_pdb("IB_res.pdb","8E8","IB","A")



Altering IB_res.pdb
Made backup of IB_res.pdb
Removed TER lines
Changed HETATM to ATOM
Renumbered atoms and residues
Changed resname from 8E8 to IB and changed chain from A to B
Connecting Atom	Old Name	New Name
C2		H21		H21
CAK		HAK1		HAK1
CAM		HAM1		HAM1
CAL		HAL1		HAL1
CAN		HAN1		HAN1
CAI		HAI1		HAI1
CAF		HAF1		HAF1
CAE		HAE1		HAE1
CAG		HAG1		HAG1
CAJ		HAJ1		HAJ1
CAA		HAA1		HAA1
CAA		HAA2		HAA2
CAD		HAD1		HAD1
CAD		HAD2		HAD2
CAO		HAO1		HAO1
CAO		HAO2		HAO2
CAP		HAP1		HAP1
CAP		HAP2		HAP2
CAQ		HAQ1		HAQ1
CAQ		HAQ2		HAQ2
CAR		HAR1		HAR1
CAR		HAR2		HAR2
NAB		HAB1		HAB1
NAB		HAB2		HAB2
CBE		HBE1		HBE1
CC3		HC31		HC31
NN1		HN11		HN11
CC4		HC41		HC41
CB		HB1		HB1
CB		HB2		HB2
CC1		HC11		HC11
CC3		HC32		HC32
CC3		HC33		HC33
NN2		HN21		HN21
CC4		HC42		HC42
CC4		HC43		HC43
Renamed hydrogens



In [13]:
#5.UNLINKED.C
#manually change desired atom name and cut and paste the line to the desired spot. 
    #eg. change added hydrogen from H26 (last one) to H00 (now first)
    
#MANUALLY CHANGE FILENAME AND RESIDUE NAMES IN THIS COMMAND 
alter_pdb("IB.pdb","8E8","IB","B")


Altering IB.pdb
Made backup of IB.pdb
Removed TER lines
Changed HETATM to ATOM
Renumbered atoms and residues
Changed resname from 8E8 to IB and changed chain from B to B
Connecting Atom	Old Name	New Name
CAA		HAA1		HAA1
CAA		HAA2		HAA2
CAD		HAD1		HAD1
NAB		HAB1		HAB1
NAB		HAB2		HAB2
CAE		HAE1		HAE1
CAF		HAF1		HAF1
CAG		HAG1		HAG1
CAI		HAI1		HAI1
CAJ		HAJ1		HAJ1
CAK		HAK1		HAK1
CAL		HAL1		HAL1
CAM		HAM1		HAM1
CAN		HAN1		HAN1
CAO		HAO1		HAO1
CAO		HAO2		HAO2
CAP		HAP1		HAP1
CAP		HAP2		HAP2
CAQ		HAQ1		HAQ1
CAQ		HAQ2		HAQ2
CAR		HAR1		HAR1
CAR		HAR2		HAR2
CBE		HBE1		HBE1
C2		H21		H21
Renamed hydrogens



### 6. Parameterize ligand

  
- Using CGenFF:
    - Ligand by itself (ligand coordinates)
        1. ...

    - Ligand with attached residue with caps
        1. ...

    - Ligand with attached residue with caps (for complete parameterization w/ hybrid atom types)
        1. Use same ligand_res mol2.
        2. "Include parameters that are already in CGenFF"
- Note: if removing lone pairs for halogens, comment out LP lines and add the LP charge to the halogen it was connected to.


### Checks for setup:
- Ligand is aligned to step3 protein pdb
- Ligand is positioned such that adding the covalent bond can be resolved by minimization (doesn't get tangled)
- Ligand is resid 1
- Atomic differences between unlinked ligand and linked ligand make sense. 
    - eg. The coordinates of the ligand hydrogen removed during linking match the coordinates of that atom in the unlinked pdb, ie the expected hydrogen was removed and this may cause the remaining hydrogens to be renamed/renumbered.

 

## Set general variables

In [14]:
#General variables

#ligand name
lig_name = "IB"

#ligand PDB (positioned correctly to prot_solvated_f)
lig_pdb = lig_name+".pdb" 

#ligand + res name
lig_res_name = "IB_res"

#ligand + res PDB
lig_res_pdb = lig_res_name+".pdb"

#ligand + res name (includes all cgenff parameters)
lig_res_allparams = "IB_res_allparams"

prot_name = "MKK7"

#resid of covalent residue on protein
cov_resid = "101" #106 if counting missing residues

patch_name = "LIGP"

#charmm-gui step3 output pdb
prot_solvated_f = "charmm-gui-5163983106/step3_pbcsetup.pdb"



## Renumber waters and change segid of waters

In [15]:
#xtal waters must have segid WATA and be TIP3

#make copy in build dir
!cp "$prot_solvated_f" "$prot_name"_solvated.pdb
print("Made file "+prot_name+"_solvated.pdb")
prot_solvated_f=prot_name+"_solvated.pdb"

#change segid
with open(prot_solvated_f, 'r') as pdbfh :
    pdbfiledata = pdbfh.read()
 # Replace the target string
pdbfiledata = pdbfiledata.replace('WATA', 'SOLV')
 # Write the file out again
with open(prot_solvated_f, 'w') as pdbfh:
    pdbfh.write(pdbfiledata)

#renumber waters
def reres_wat(pdbfile):
    tempfile=pdbfile+".temp"
    tempfh = open(tempfile,"w+")
    with open(pdbfile, 'r') as pdbfh :
        lines = pdbfh.readlines()
    resnbr=0
    prev_res=None
    for line in lines:
        if line.startswith('ATOM'):
            if line[72:76] == 'SOLV':
                if prev_res != line[22:27]:
                    resnbr=resnbr+1
                prev_res=line[22:27]
                line = line[:22]+str(resnbr).rjust(4).ljust(5)+line[27:]



        tempfh.write(line)    
    tempfh.close()
    !mv $tempfile $pdbfile
    return

reres_wat(prot_solvated_f)

Made file MKK7_solvated.pdb


## Segment system 

In [16]:
#get segments
#CHANGE ION SELECTIONS IF USING ANYTHING OTHER THAN K and Cl
os.makedirs("segments", exist_ok=True)

segmenter='''
mol addfile ../PROT_SOLVATED_F

set protein_residues [atomselect top "protein"]
set water_residues [atomselect top "water"]
set pot_residues [atomselect top "resname POT"]
set cla_residues [atomselect top "resname CLA"]

#separate every 10k waters if the residue number doesnt go to 5 digits
#this gets around 10k res limit/duplicate segid's for solvent but isnt needed atm
#mol addfile segments/solv.pdb
#set waterresidues [atomselect top "water"]
#set newsegnames [list]
#set postfix 0
#for { set i 0 } { $i < [llength $waterresidues] } { incr i } {
#    if { $i % 30000 == 0 } {
#        incr postfix
#    }
#    lappend newsegnames W$postfix
#}
#$waterresidues set segname $newsegnames



#change chain names (ligand will be B)
$protein_residues set chain A
$water_residues set chain C
$pot_residues set chain D
$cla_residues set chain E

$protein_residues writepdb proa.pdb
$water_residues writepdb solv.pdb
$pot_residues writepdb pot.pdb
$cla_residues writepdb cla.pdb

exit

'''
segmenter = re.sub("PROT_SOLVATED_F", prot_solvated_f, segmenter)



with open("./segments/segmenter.tcl", "w") as segmenter_fh:
    segmenter_fh.write(segmenter)
    
!cp $lig_pdb './segments/'$lig_name'.pdb' 

md('''
#### Paste the following code in VMD Tcl console to segment {0}

```tcl
cd {1}/segments
source segmenter.tcl
```

##### or run the following cell to run the vmd script from command line:
'''.format(prot_solvated_f, os.getcwd()))


#### Paste the following code in VMD Tcl console to segment MKK7_solvated.pdb

```tcl
cd /mnt/c/Users/Kyle/Desktop/Roux/kininhibs/main/MKK7_IB/build/segments
source segmenter.tcl
```

##### or run the following cell to run the vmd script from command line:


In [17]:
os.chdir('segments')
!vmd -dispdev text -e segmenter.tcl
os.chdir('..')

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 7003MB (42%)
Info) No CUDA accelerator devices available.
Info) Dynamically loaded 3 plugins in directory:
Info) /usr/local/lib/vmd/plugins/LINUXAMD64/molfile
ERROR) Illegal rendering mode: GLSL
after#0
after#1
/usr/local/lib/vmd/scripts/tcl8.5 /usr/local/lib/vmd/scripts /usr/local/lib/lib /Projects/johns/tcl/8.5.6/lib_LINUXAMD64/lib /usr/local/lib/vmd/scripts/vmd /usr/local/lib/vmd/plugins/LINUXAMD64/tcl /

#### Checks:
- PDBs are ok  

In [89]:
# check segmented pdbs

check='''
set env(MYARG) "segments/*.pdb"

set filelist [glob $env(MYARG)]

foreach i $filelist {
  echo " "
  echo "Loading $i"
  mol new "$i" type {pdb} waitfor all
  echo " "
}  
'''

with open("./segments/check.tcl", "w") as check_fh:
    check_fh.write(check)
!vmd -e segments/check.tcl

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 6715MB (41%)
Info) No CUDA accelerator devices available.
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
Info) OpenGL renderer: NVIDIA GeForce GTX 1070 Ti/PCIe/SSE2
Info)   Features: STENCIL MSAA(4) MDE MTX NPOT PP PS 
Info)   GLSL rendering mode is NOT available.
Info)   Textures: 2-D (32768x32768), 3-D (16384x16384x16384), Multitexture (4)
Info) Dynamically 

## Separate .rtf and .prm chunks from .str's

In [18]:
def split_str(f_nosuffix):
    with open(f_nosuffix+".str","r") as str_file,open(f_nosuffix+".rtf", 'w') as rtf_file,open(f_nosuffix+".prm", 'w') as prm_file:
        rtfmode=False
        prmmode=False
        for line in str_file:
            if "read rtf card append" in line:
                rtfmode=True
                prmmode=False
            if "read param card flex append" in line:
                rtfmode=False
                prmmode=True
            if rtfmode:
                rtf_file.write(line)
            elif prmmode:
                prm_file.write(line)
            else:
                continue
        rtf_file.write("\n"+"RETURN")
    return

In [19]:
#run the function here. use filename without the suffix
split_str(lig_name)
split_str(lig_res_name)
split_str(lig_res_allparams)

## Find atoms with different atom types between two .rtf files

In [20]:
f1=lig_res_name+".rtf" #can change variable
f2=lig_name+".rtf" #can change variable


fh1=open(f1,"r")
fh2=open(f2, 'r')
lines1 = fh1.readlines()
lines2 = fh2.readlines()
print("ATOM <pdb name> <atom type in f1> <atom type in f2>")
for line1 in lines1:
    #print(line1[19:25])
    for line2 in lines2:
        if "ATOM" in line1 and "ATOM" in line2:
            if line1[5:12].strip() == line2[5:12].strip():
                if line1[12:18] != line2[12:18]:
                    print(line1[0:18]+" "+line2[12:18])

ATOM <pdb name> <atom type in f1> <atom type in f2>
ATOM CAA    CG321  CG2DC3
ATOM CAD    CG321  CG2DC1
ATOM HAA1   HGA2   HGA5  
ATOM HAA2   HGA2   HGA5  
ATOM HAD1   HGA2   HGA4  


## Establish atom type dictionary and apply it to misc files if needed

In [21]:
#dictionary for switching atom types
#determined by which atoms will be found the protein side in the patch for dihedral terms 


cgenff_charmm_dict = {"CG321":"CT2", #CB
                     "HGA2":"HA2",
                     "CG311":"CT1"} #CA
#                     "NG311":"NZ   ",


#adjust for spaces
cgenff_charmm_dict = {key.ljust(6): value.ljust(6) for key, value in cgenff_charmm_dict.items()}

print(cgenff_charmm_dict)

{'CG321 ': 'CT2   ', 'HGA2  ': 'HA2   ', 'CG311 ': 'CT1   '}


In [22]:
#sub atom types in existing files 
def rpl_atomtypes(filename):     
    # open the source file and read it        
    fh = open(filename, 'r')
    subject = fh.read()
    fh.close()
    #replace
    for k,v in cgenff_charmm_dict.items():
        subject = re.sub(k, v, subject)
    # write the file
    f_out = open(filename, 'w')
    f_out.write(subject)
    f_out.close()
    return

In [175]:
#not needed 
#rpl_atomtypes(lig_name+".rtf")
#rpl_atomtypes(lig_res_name+".rtf")

## Check charges from an .rtf file

In [24]:
#this cell adds up the charges of a molecule from an rtf file
#f=lig_name+".rtf" #user assign
f=lig_res_name+".rtf" #user assign

fh=open(f,"r")
lines = fh.readlines()
charge_list=[]
print('Atom \t\tCharge')
for line in lines:
    if "ATOM" in line:
        print(line[5:12],'\t',line[19:25])
        charge_list.append(float(line[19:25]))
print("Total:",round(sum(charge_list),3))
         

Atom 		Charge
CAA     	 -0.068
CAD     	 -0.106
CAE     	 -0.115
CAF     	 -0.115
CAG     	 -0.115
CAI     	 -0.115
CAJ     	 -0.115
CAK     	 -0.118
CAL     	 -0.118
CAM     	 -0.117
CAN     	 -0.117
CAO     	 -0.152
CAP     	 -0.155
CAQ     	  0.147
CAR     	  0.065
CAW     	  0.470
CAY     	  0.216
CAZ     	  0.216
NAB     	 -0.765
NAU     	 -0.674
OAC     	 -0.533
OAV     	 -0.432
CBA     	  0.064
CBB     	  0.193
CBE     	  0.159
NBF     	  0.014
NBG     	 -0.660
N1      	 -0.741
C2      	  0.498
N3      	 -0.794
C4      	  1.010
C5      	 -0.388
C6      	  0.483
HAA1    	  0.090
HAA2    	  0.090
HAB1    	  0.378
HAB2    	  0.378
HAD1    	  0.090
HAD2    	  0.090
HAE1    	  0.115
HAF1    	  0.115
HAG1    	  0.115
HAI1    	  0.115
HAJ1    	  0.115
HAK1    	  0.115
HAL1    	  0.115
HAM1    	  0.115
HAN1    	  0.115
HAO1    	  0.090
HAO2    	  0.090
HAP1    	  0.090
HAP2    	  0.090
HAQ1    	  0.090
HAQ2    	  0.090
HAR1    	  0.090
HAR2    	  0.090
HBE1    	  0.090
H21     	  0.128


## Find atoms with different charges between two .rtf files

In [25]:
f1=lig_res_name+".rtf"
f2=lig_name+".rtf"

fh1=open(f1,'r')
fh2=open(f2, 'r')
lines1 = fh1.readlines()
lines2 = fh2.readlines()
chargediff=0
print("ATOM <pdb name> <atom type in f1> <charge in f1> ! <charge in f2>")
for line1 in lines1:
    #print(line1[19:25])
    for line2 in lines2:
        if "ATOM" in line1 and "ATOM" in line2:
            if line1[5:12].strip() == line2[5:12].strip():
                if line1[19:25] != line2[19:25]:
                    print(line1[0:27]+" "+line2[19:25])
                    chargediff=chargediff+(round(float(line1[19:25]),3)-round(float(line2[19:25]),3))
print("total charge difference ([f1] - [f2])=",round(chargediff,3))

ATOM <pdb name> <atom type in f1> <charge in f1> ! <charge in f2>
ATOM CAA    CG321  -0.068 ! -0.421
ATOM CAD    CG321  -0.106 ! -0.154
ATOM CAW    CG2O1   0.470 !  0.495
ATOM OAC    OG2D1  -0.533 ! -0.514
ATOM NBG    NG2S0  -0.660 ! -0.624
ATOM HAA1   HGA2    0.090 !  0.210
ATOM HAA2   HGA2    0.090 !  0.210
ATOM HAD1   HGA2    0.090 !  0.150
total charge difference ([f1] - [f2])= 0.021


## Find different parameters for the same atomtypes between two .prm files
#### - Note that there could be multiple dihedral lines for the same four atoms. This cell assumes that patches wont change multiplicity in the dihedral term 

In [26]:
f1=lig_res_name+".prm"
f2=lig_name+".prm"

fh1=open(f1,'r')
fh2=open(f2, 'r')
lines1 = fh1.readlines()
lines2 = fh2.readlines()
seg1=None
seg2=None

for line1 in lines1:
    if "BONDS" in line1:
        seg1="BONDS"
        print(seg1)
    elif "ANGLES" in line1:
        seg1="ANGLES"
        print(seg1)
    elif "DIHEDRALS" in line1:
        seg1="DIHEDRALS"
        print(seg1)
    elif "IMPROPERS" in line1:
        seg1="IMPROPERS"
        print(seg1)
    for line2 in lines2:
        line1 = line1.split("!")[0] #take out comment
        line2 = line2.split("!")[0] #take out comment
        if "BONDS" in line1:
            seg2="BONDS"
        elif "ANGLES" in line2:
            seg2="ANGLES"
        elif "DIHEDRALS" in line2:
            seg2="DIHEDRALS"
        elif "IMPROPERS" in line2:
            seg2="IMPROPERS"
        if seg1==seg2=="BONDS":
            if line1[0:13] == line2[0:13]:
                if line1[13:] != line2[13:]:
                    print('{0}:\n{1}'.format(f1,line1))
                    print('{0}:\n{1}'.format(f2,line2))
                    print('\n')
        elif seg1==seg2=="ANGLES":
            if line1[0:20] == line2[0:20]:
                if line1[20:] != line2[20:]:
                    print('{0}:\n{1}'.format(f1,line1))
                    print('{0}:\n{1}'.format(f2,line2))
                    print('\n')
        elif seg1==seg2=="DIHEDRALS":
            if line1[0:27] == line2[0:27] and line1[40:41] == line2[40:41]: 
                if line1[27:] != line2[27:]:
                    print('{0}:\n{1}'.format(f1,line1))
                    print('{0}:\n{1}'.format(f2,line2))
                    print('\n')
        elif seg1==seg2=="IMPROPERS":
            if line1[0:27] == line2[0:27]:
                if line1[27:] != line2[27:]:
                    print('{0}:\n{1}'.format(f1,line1))
                    print('{0}:\n{1}'.format(f2,line2))
                    print('\n')


BONDS
ANGLES
DIHEDRALS
IMPROPERS


## Determine the amount of charge needed to get back to integer charge
- Note that chargediff represents the extra charge added by patch ie the charge that should be subtracted from another atom in fudging
- This is considering the charges before and after patching ie before and after writing the patch .rtf below. Therefore it may be easier to write the structure of the patch .rtf first, come back to this second, and finalize the patch .rtf third.


In [31]:
#syntax: <atom name> = <charge after patch> - <charge before patch>

chargediff=0

#add differences in charge of affected protein atoms 
#Comparing to CYS from top_all36_prot.rtf
#chargediff += (-0.454) - (-0.47) #n (nh1)
#chargediff += (0.296) - (0.31) #hn (h)
#chargediff += (0.072) - (0.07) #ca (ct1)
#chargediff += (0.09) - (0.09) #ha (hb1)
chargediff += (-0.051) - (-0.11) #cb (ct2)
#chargediff += (0.09) - (0.09) #hb1 (ha2)
#chargediff += (0.09) - (0.09) #hb2 (ha2)
chargediff += (-0.246) - (-0.23) #sg (s)
#chargediff += (0.474) - (0.51) #c (c)
#chargediff += (-0.519) - (-0.51) #o (o)

#add differences in charge of affected ligand atoms 
chargediff += (-0.068) - (-0.421) #caa
chargediff += (-0.106) - (-0.154) #cad
chargediff += (0.470) - (0.495) #caw
chargediff += (-0.533) - (-0.514) #oac
chargediff += (-0.660) - (-0.624) #nbg
chargediff += (0.090) - (0.210) #haa1
chargediff += (0.090) - (0.210) #haa2
chargediff += (0.090) - (0.150) #had1
chargediff += (-0.426) - (-0.432) #oav (fudging)

#subtract charge of deleted protein atoms 
chargediff -= (0.16) #hg1

#subtract charge of deleted ligand atoms 
#chargediff -= () #

#add charge of added protein atoms 
#chargediff += () #

#add charge of added ligand atoms 
chargediff += (0.090) #had2

#add charge of added solvent atoms 
#chargediff += (1) #k

#subtract charge of deleted solvent atoms 
#chargediff -= (-1) #cl

print(round(chargediff,3))


-0.0


## Write the patch .rtf
- Add number prefixes to atom names (1 or 2 in the order given in the psfgen command and fix atom types to reflect how they are in the linked state)
- Add fudged charges 

In [21]:
#calculate IC lines
#IMPORTANT: when adding IC lines to replace coordinates of an existing atom, 
    #you need to add a DELETE ATOM line to the patch rtf for atom1 in the IC line
    #AND 
    #you need to add a BOND line for the first two atoms in the IC line since the bond was deleted too 

pdbfile=lig_res_name+'.pdb'
#atomnames should be without number prefix (put them in after) and be listed in order as they will come out on the IC line and be 4 char including spaces. Generate IC lines for added or spatially-altered atoms
atom1='HAD2'
atom2='CAD '
atom3='CAA '
atom4='SG '

with open(pdbfile, 'r') as pdbfh :
    lines = pdbfh.readlines()
for line in lines:
    if line.startswith('ATOM'):
        if atom1.strip()==line[11:16].strip():
            a1_x=float(line[30:38].strip())
            a1_y=float(line[38:46].strip())
            a1_z=float(line[46:54].strip())
        elif atom2.strip()==line[11:16].strip():
            a2_x=float(line[30:38].strip())
            a2_y=float(line[38:46].strip())
            a2_z=float(line[46:54].strip())
        elif atom3.strip()==line[11:16].strip():
            a3_x=float(line[30:38].strip())
            a3_y=float(line[38:46].strip())
            a3_z=float(line[46:54].strip())
        elif atom4.strip()==line[11:16].strip():
            a4_x=float(line[30:38].strip())
            a4_y=float(line[38:46].strip())
            a4_z=float(line[46:54].strip())

def calculateDistance(x1,y1,z1,x2,y2,z2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return dist

def calculateAngle(x1,y1,z1,x2,y2,z2,x3,y3,z3):
    a = np.array([x1, y1, z1])
    b = np.array([x2, y2, z2])
    c = np.array([x3, y3, z3])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def calculateDihedral(p):
    #praxeolitic formula
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

atom_ar = np.array([
                [ a1_x, a1_y, a1_z ],
                [ a2_x, a2_y, a2_z ],
                [ a3_x, a3_y, a3_z ],
                [ a4_x, a4_y, a4_z ]
                ])

IC1=round(calculateDistance(a1_x,a1_y,a1_z,a2_x,a2_y,a2_z),4)
IC2=round(calculateAngle(a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,a3_x,a3_y,a3_z),4)
IC3=round(calculateDihedral(atom_ar),4)
IC4=round(calculateAngle(a2_x,a2_y,a2_z,a3_x,a3_y,a3_z,a4_x,a4_y,a4_z),4)
IC5=round(calculateDistance(a3_x,a3_y,a3_z,a4_x,a4_y,a4_z),4)
print('IC {0} {1} {2} {3}'.format(atom1,atom2,atom3,atom4)
      +'{: 8.4f}'.format(IC1)
      +'{: 9.4f}'.format(IC2)
      +'{: 10.4f}'.format(IC3)
      +'{: 9.4f}'.format(IC4)
      +'{: 8.4f}'.format(IC5))


IC HAD2 CAD  CAA  SG   1.0894 109.7617  -96.2506 120.2539  1.6055


In [22]:

header = '''read rtf card append
* Topologies generated by
* CHARMM General Force Field (CGenFF) program version 2.5
*
36 1

! "penalty" is the highest penalty score of the associated parameters.
! Penalties lower than 10 indicate the analogy is fair; penalties between 10
! and 50 mean some basic validation is recommended; penalties higher than
! 50 indicate poor analogy and mandate extensive validation/optimization.
'''

#use https://pages.jh.edu/rschlei1/Random_stuff/publications/charmmbook.pdf for IC usage
#assign protein atoms charmm atom types except for the linking atom (eg sulfur in cysteine), which stays cgenff atomtype

patch_rtf = '''
PRES PATCHNAME		0.00
ATOM 1CB     CT2    -0.051 ! -0.11 from top_all36_prot.rtf RESI CYS
ATOM 1SG     SG311  -0.246 ! -0.23 from top_all36_prot.rtf RESI CYS
ATOM 2CAA    CG321  -0.068 ! -0.421, was CG2DC3
ATOM 2CAD    CG321  -0.106 ! -0.154, was CG2DC1
ATOM 2CAW    CG2O1   0.470 !  0.495
ATOM 2OAC    OG2D1  -0.533 ! -0.514
ATOM 2NBG    NG2S0  -0.660 ! -0.624
ATOM 2HAA1   HGA2    0.090 !  0.210, was HGA5
ATOM 2HAA2   HGA2    0.090 !  0.210, was HGA5
ATOM 2HAD1   HGA2    0.090 !  0.150, was HGA4
ATOM 2OAV    OG301  -0.426 ! -0.432, fudged 
ATOM 2HAD2   HGA2    0.090 ! added

BOND 1SG   2CAA

BOND 2HAD1 2CAD
BOND 2HAD2 2CAD
BOND 2HAA1 2CAA
BOND 2HAA2 2CAA

DELETE ATOM 1HG1 
DELETE ATOM 2HAA1
DELETE ATOM 2HAA2
DELETE ATOM 2HAD1

IC 2HAD1 2CAD 2CAA 1SG    1.0901 110.0353   23.4104 120.2539  1.6055
IC 2HAD2 2CAD 2CAA 1SG    1.0894 109.7617  -96.2506 120.2539  1.6055
IC 2HAA1 2CAA 2CAD 2CAW   1.0900 103.1338 -102.2136 108.4486  1.5025
IC 2HAA2 2CAA 2CAD 2CAW   1.0908 106.7508   22.1977 108.4486  1.5025

END
RETURN'''

patch_rtf = re.sub("PATCHNAME", patch_name, patch_rtf)


#for k,v in cgenff_charmm_dict.items():
#    patch_rtf = re.sub(k, v, patch_rtf)

with open(patch_name+".rtf","w") as patch_rtf_file:
    patch_rtf_file.write(header)
    patch_rtf_file.write(patch_rtf)

## Write the patch .prm
- Acceptable to have extra entries in patch prm file (they won't be used)
- Take linked prm file with all dihedrals but those dihedrals have cgenff names so you need to replace names
- Add new bonds, angles, and dihedrals with new names

In [ ]:
#prm file of lig+res
lig_res_prm_file = lig_res_name+".prm" 

#prm file of lig+res that includes all cgenff terms 
lig_res_prm_allparams_file = lig_res_allparams+".prm" 

#new atoms or changed atomtypes from patch.rtf (cgenff names). Any line from <lig_res>.prm 
    #that has any of these will be included.
atypelist1=["SG311","CG321","HGA2"]

#Atom(s) that would require a line with hybridized atomtypes (eg. SG311 in a cysteine linkage 
    #bc there should be charmm atomtypes on one side of the SG311 and cgenff atomtypes on the other)
atypelist2=["SG311"]

header = '''read param card flex append
* Parameters generated by analogy by
* CHARMM General Force Field (CGenFF) program version 2.5
*

! Penalties lower than 10 indicate the analogy is fair; penalties between 10
! and 50 mean some basic validation is recommended; penalties higher than
! 50 indicate poor analogy and mandate extensive validation/optimization.

'''

#WILL NEED TO MANUALLY :
    # - ADD THE PARAMS DECLARED AS MISSING BY CHAMBER AND MANUALLY CHANGE THE ATOMTYPES TO 
        #MATCH THE MISSING PARAM'D ATOMS
    # - CHECK REPLACED PARAMS AND REMOVE IF NECESSARY 

#general rules for choosing b/w cgenff and prot params in the event of missing params:
    #dihedral: if 3 atom types are protein (eg CT2 instead of CG321), use protein. Otherwise, cgenff. 
    #angles and impropers: if central atom is protein, use protein. Otherwise, cgenff
    #bonds: cgenff
    #if params dont specifically exist in prot, use cgenff


#angle format ex
#####  #####  #####   ###.##    ###.## 
#dihedral format ex 
#####  #####  #####  #####    ###.####  #   ###.####

extra_bonds = '''
SG311    CT2   198.000     1.8180 ! PROT fitted to C-S s   9/26/92 (FL), (patched s)-(patched cb) 
'''
extra_angles = '''
SG311  CT2    CT1      58.000  112.5000 ! (patched s)-(patched cb)-ca, from par_all36m_prot.prm (same in cgenff)
CT2    SG311  CG321    34.00    95.00 ! (patched cb)-(patched s)-caa, from par_all36_cgenff.prm (same in cgenff)
SG311  CT2    HA2      46.10    111.30 ! from par_all36m_prot.prm (same in cgenff)
'''


extra_dihedrals = '''
C      CT1    CT2    SG311      0.2400  1   180.00  !fitted cys params with sg311 nomenclature from par_all36m_prot.prm
C      CT1    CT2    SG311      0.7500  2   180.00
C      CT1    CT2    SG311      1.3500  3   180.00
NH1    CT1    CT2    SG311      0.3400  1     0.00
NH1    CT1    CT2    SG311      0.5000  2   180.00
NH1    CT1    CT2    SG311      1.4300  3     0.00
HB1    CT1    CT2    SG311      0.2000  3     0.00 ! From X    CT1  CT2  X


CT1    CT2    SG311  CG321      0.2400  1   180.00 ! IB_res , from CG321 CG321 SG311 CG321, PENALTY= 0.6
CT1    CT2    SG311  CG321      0.3700  3     0.00 ! IB_res , from CG321 CG321 SG311 CG321, PENALTY= 0.6
CG321  CG321  SG311  CT2        0.2400  1   180.00 ! PROT expt. MeEtS,      3/26/92 (FL)
CG321  CG321  SG311  CT2        0.3700  3     0.00 ! PROT expt. MeEtS,      3/26/92 (FL)
HGA2   CG321  SG311  CT2        0.2800  3     0.00 ! PROT DTN 8/24/90
HA2    CT2    SG311  CG321      0.2800  3     0.00 ! PROT DTN 8/24/90
'''
    

with open(lig_res_prm_file, 'r') as f, open(patch_name+".prm", 'w') as patch_prm_file:
    patch_prm_file.write(header)
    for line in f:
        if "BONDS" in line:
            patch_prm_file.write("\n"+line)
            seg="BONDS"
        elif "ANGLES" in line:
            patch_prm_file.write(extra_bonds)
            patch_prm_file.write("\n"+line)
            seg="ANGLES"
        elif "DIHEDRALS" in line:
            patch_prm_file.write(extra_angles)
            patch_prm_file.write("\n"+line)
            seg="DIHEDRALS"
        elif "IMPROPERS" in line:
            patch_prm_file.write(extra_dihedrals)
            patch_prm_file.write("\n"+line)
            seg="IMPROPERS"
        if any(word in line for word in ["END","RETURN"]):
                patch_prm_file.write("\n"+line)     
        orig_line = line
        line = line.split("!")[0]+"\n"
        
        #this will make duplicate lines between lig_prm_file and patch_prm_file, but parmed does not double energy
        if any(word in line for word in atypelist1):
            patch_prm_file.write(line)
            
print('Lines you may want to add or permutate manually with hybrid atomtypes:')
with open(lig_res_prm_allparams_file, 'r') as f:
    for line in f:
        if "BONDS" in line:
            seg="BONDS"
            print('\nBONDS')
        elif "ANGLES" in line:
            seg="ANGLES"
            print('\nANGLES')
        elif "DIHEDRALS" in line:
            seg="DIHEDRALS"
            print('\nDIHEDRALS')
        elif "IMPROPERS" in line:
            seg="IMPROPERS"
            print('\nIMPROPERS')
        orig_line = line
        line = line.split("!")[0]+"\n"
        if any(word in line for word in atypelist2): 
            print(orig_line.strip("\n"))

#### Checks:
- Make sure ligand resname is the same for the ligand .rtf and .pdb file(s). 
    - .rtf resname probably needs to be changed from the cgenff default.

## Write the extra .prm
- Contains modified terms from previous scans
- Contains dummy terms 

In [2]:
#may change if cgenff prm is updated. These use jul21 version
header = '''read param card flex append
* Parameters generated by analogy by
* CHARMM General Force Field (CGenFF) program version 2.5
*

! Penalties lower than 10 indicate the analogy is fair; penalties between 10
! and 50 mean some basic validation is recommended; penalties higher than
! 50 indicate poor analogy and mandate extensive validation/optimization.

'''

#angle format ex
#####  #####  #####   ###.##    ###.## 
#dihedral format ex 
#####  #####  #####  #####    ###.####  #   ###.####

extra_bonds = '''
SG3O2  FGP1    185.00     1.6200 ! O44 , from CG321 SG3O2, penalty= 360, was 1.79 corrected to QM opt, -trayder
'''
extra_angles = '''
'''
extra_dihedrals = '''
CG321  CG311  NG2R51 NG2R50     0.7550  1     90.00 ! frag1
CG321  CG311  NG2R51 NG2R50     0.5700  2    300.00 ! frag1
CG321  CG311  NG2R51 NG2R50     0.0450  3     60.00 ! frag1
CG321  CG311  NG2R51 NG2R50     0.5900  4      0.00 ! frag1

! CG2DC3 CG2DC1 CG2O1  OG2D1      1.0000  1      0.00 ! frag6
! CG2DC3 CG2DC1 CG2O1  OG2D1      0.8000  2    180.00 ! frag6
! CG2DC3 CG2DC1 CG2O1  OG2D1      0.0300  3    180.00 ! frag6
! CG2DC3 CG2DC1 CG2O1  OG2D1      0.1300  4      0.00 ! frag6

CG321 CG321 CG2O1  OG2D1     -2.0500  1    180.00 ! frag8
CG321 CG321 CG2O1  OG2D1     -0.9000  2      0.00 ! frag8
CG321 CG321 CG2O1  OG2D1     -0.2500  3    180.00 ! frag8
CG321 CG321 CG2O1  OG2D1     -0.1500  4    180.00 ! frag8

CG321 CG1T1 CG1T2 HGPAM1        0.0     1      0.0  ! term alkyne
NG2S1 CG321 CG1T1 CG1T2         0.0     1      0.0  ! term alkyne
HGA2  CG321 CG1T1 CG1T2         0.0     1      0.0  ! term alkyne

NG1T1  CG1N1  CG2DC1 CG2DC1       0.0     1      0.0  ! cyano
NG1T1  CG1N1  CG2DC1 CG2O1        0.0     1      0.0  ! cyano
NG1T1  CG1N1  CG311  CG311        0.0     1      0.0  ! cyano
NG1T1  CG1N1  CG311  CG2O1        0.0     1      0.0  ! cyano
NG1T1  CG1N1  CG311  HGA1         0.0     1      0.0  ! cyano

NG2R62 CG2R64 NG301  CG321      1.9800  2   180.00 ! From NG2R62 CG2R64 NG301  CG331 in cgenff (TMC, yxu) to replace the flavin piperazine dihedral (NG2R62 CG2R64 NG301  CG321      0.4200  2     0.00) 
'''
extra_impropers = '''
'''
extra_nbfix = '''
'''

with open("extra"+'.prm', 'w') as extra_prm_fh:
    extra_prm_fh.write(header)
    extra_prm_fh.write('\nBONDS\n')
    extra_prm_fh.write(extra_bonds)
    extra_prm_fh.write('\nANGLES\n')
    extra_prm_fh.write(extra_angles)
    extra_prm_fh.write('\nDIHEDRALS\n')
    extra_prm_fh.write(extra_dihedrals)
    extra_prm_fh.write('\nIMPROPERS\n')
    extra_prm_fh.write(extra_impropers)
    extra_prm_fh.write('\nNBFIX\n')
    extra_prm_fh.write(extra_nbfix)
    extra_prm_fh.write('\nEND\n')
    extra_prm_fh.write('\nRETURN\n')


## Construct .psf's with psfgen in VMD's Tcl console
- Names might have changed between versions so the pdbalias lines might be screwed
- Make sure toppar exists with the required files
- Terminal patches might need to be altered (eg glycine N-term needs GLYP, not NTER)
- Balance charge that may have been changed between linked/unlinked by removing solvent ions. 
    - delatom <segid> [resid] [atomname]
    - eg. "delatom CLA 1" if a charge is lost in linked state (eg LYS is neutralized)
- Guessed coords in unlinked may imply bad MODELLER output with overlapping side chains. Use pymol to sculpt that side chain to a better spot (delete any non-xtal interfereing waters from prot_solvated_f) and replace the coords in CHARMM-prot_solvated_f. Then change prot_solvated_f to the new pdb, and do the cells that use this pdb again such as segments and psf generation. Non-clashed coords for the interfered side chains can maybe be found in the CHARMM-GUI step 2 pdb.

In [3]:
os.makedirs("toppar", exist_ok=True)
!mv $patch_name".rtf" toppar/
!mv $patch_name".prm" toppar/
!mv $lig_name".rtf" toppar/
!mv $lig_name".prm" toppar/
!mv "extra.prm" toppar/

NameError: name 'os' is not defined

In [101]:
!cp ../../resources/toppar_c36_jul21_essential/* toppar/

### a) Unlinked

In [24]:
pgn_contents_u = '''
package require psfgen
resetpsf

topology ./toppar/top_all36_prot.rtf
#topology ./toppar/par_all36m_prot.prm
topology ./toppar/top_all36_cgenff.rtf
#topology ./toppar/par_all36_cgenff.prm

topology ./toppar/toppar_water_ions.str

topology ./toppar/LIGNAME.rtf
topology ./toppar/LIGNAME.prm

pdbalias residue HOH TIP3
pdbalias atom TIP3 O OH2
pdbalias residue HIS HSD

segment PROA {
    first NTER
    last CTER
    pdb ./segments/proa.pdb
}
coordpdb ./segments/proa.pdb PROA

segment LIG {pdb ./segments/LIGNAME.pdb}
coordpdb ./segments/LIGNAME.pdb LIG

segment SOLV {
    auto none
    pdb ./segments/solv.pdb
}
coordpdb ./segments/solv.pdb SOLV

segment POT {pdb ./segments/pot.pdb}
coordpdb ./segments/pot.pdb POT

segment CLA {pdb ./segments/cla.pdb}
coordpdb ./segments/cla.pdb CLA

regenerate angles dihedrals 
guesscoord

writepdb PROTNAME_LIGNAME_u.pdb
writepsf x-plor PROTNAME_LIGNAME_u.psf

echo " "
mol new PROTNAME_LIGNAME_u.pdb type {pdb} waitfor all
mol addfile PROTNAME_LIGNAME_u.psf type {psf} waitfor all
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
echo "Net charge: $charge"
echo " "

exit
'''
pgn_contents_u = re.sub("LIGNAME", lig_name, pgn_contents_u)
pgn_contents_u = re.sub("PROTNAME", prot_name, pgn_contents_u)

with open(prot_name+"_"+lig_name+"_u.pgn", "w") as pgn_fh:
    pgn_fh.write(pgn_contents_u)

md('''
#### Paste the following code in VMD Tcl console to generate the .psf 
#### or run the next cell

```tcl
cd {0}
source {1}.pgn
```


#### After loading *only* the pdb and psf into VMD...
   - This gets the net charge of the psf:
```tcl
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
```


   - This gets the charge of specific selection (eg. protein):
```tcl
puts [vecsum [[atomselect top "protein"] get charge]]
```


   - This writes a crd file:
```tcl
[atomselect top all] writecrd {1}.crd
```

'''.format(os.getcwd(),prot_name+"_"+lig_name+"_u"))



#### Paste the following code in VMD Tcl console to generate the .psf 
#### or run the next cell

```tcl
cd /mnt/c/Users/Kyle/Desktop/Roux/kininhibs/main/MKK7_IB/build
source MKK7_IB_u.pgn
```


#### After loading *only* the pdb and psf into VMD...
   - This gets the net charge of the psf:
```tcl
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
```


   - This gets the charge of specific selection (eg. protein):
```tcl
puts [vecsum [[atomselect top "protein"] get charge]]
```


   - This writes a crd file:
```tcl
[atomselect top all] writecrd MKK7_IB_u.crd
```



In [25]:
!vmd -dispdev text -e "$prot_name"_"$lig_name"_u.pgn

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 7007MB (42%)
Info) No CUDA accelerator devices available.
Info) Dynamically loaded 3 plugins in directory:
Info) /usr/local/lib/vmd/plugins/LINUXAMD64/molfile
ERROR) Illegal rendering mode: GLSL
after#0
after#1
/usr/local/lib/vmd/scripts/tcl8.5 /usr/local/lib/vmd/scripts /usr/local/lib/lib /Projects/johns/tcl/8.5.6/lib_LINUXAMD64/lib /usr/local/lib/vmd/scripts/vmd /usr/local/lib/vmd/plugins/LINUXAMD64/tcl /

### b) Linked

In [26]:

pgn_contents_l = '''
package require psfgen
resetpsf

topology ./toppar/top_all36_prot.rtf
#topology ./toppar/par_all36m_prot.prm
topology ./toppar/top_all36_cgenff.rtf
#topology ./toppar/par_all36_cgenff.prm


topology ./toppar/toppar_water_ions.str

topology ./toppar/LIGNAME.rtf
#topology ./toppar/LIGNAME.prm
topology ./toppar/PATCHNAME.rtf
topology ./toppar/PATCHNAME.prm

pdbalias residue HOH TIP3
pdbalias atom TIP3 O OH2
pdbalias residue HIS HSD

segment PROA {
    first NTER
    last CTER
    pdb ./segments/proa.pdb
}
coordpdb ./segments/proa.pdb PROA

segment LIG {pdb ./segments/LIGNAME.pdb}
coordpdb ./segments/LIGNAME.pdb LIG

segment SOLV {
    auto none
    pdb ./segments/solv.pdb
}
coordpdb ./segments/solv.pdb SOLV

segment POT {pdb ./segments/pot.pdb}
coordpdb ./segments/pot.pdb POT

segment CLA {pdb ./segments/cla.pdb}
coordpdb ./segments/cla.pdb CLA

patch PATCHNAME PROA:PROTRESID LIG:1
regenerate angles dihedrals 
guesscoord

writepdb PROTNAME_LIGNAME_l.pdb
writepsf x-plor PROTNAME_LIGNAME_l.psf

echo " "
mol new PROTNAME_LIGNAME_l.pdb type {pdb} waitfor all
mol addfile PROTNAME_LIGNAME_l.psf type {psf} waitfor all
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
echo "Net charge: $charge"
echo " "

exit
'''
pgn_contents_l = re.sub("LIGNAME", lig_name, pgn_contents_l)
pgn_contents_l = re.sub("PROTNAME", prot_name, pgn_contents_l)
pgn_contents_l = re.sub("PROTRESID", cov_resid, pgn_contents_l)
pgn_contents_l = re.sub("PATCHNAME", patch_name, pgn_contents_l)

with open(prot_name+"_"+lig_name+"_l.pgn", "w") as pgn_fh:
    pgn_fh.write(pgn_contents_l)

md('''
#### Paste the following two lines in VMD Tcl console to generate the .psf
#### or run the next cell

```tcl
cd {0}
source {1}.pgn
```


#### After loading *only* the pdb and psf into VMD...
   - This gets the net charge of the psf:
```tcl
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
```


   - This gets the charge of specific selection (eg. protein):
```tcl
puts [vecsum [[atomselect top "protein"] get charge]]
```


   - This writes a crd file:
```tcl
[atomselect top all] writecrd {1}.crd
```

'''.format(os.getcwd(),prot_name+"_"+lig_name+"_l"))



#### Paste the following two lines in VMD Tcl console to generate the .psf
#### or run the next cell

```tcl
cd /mnt/c/Users/Kyle/Desktop/Roux/kininhibs/main/MKK7_IB/build
source MKK7_IB_l.pgn
```


#### After loading *only* the pdb and psf into VMD...
   - This gets the net charge of the psf:
```tcl
set sel [atomselect top all]
set charge [vecsum [$sel get charge]]
```


   - This gets the charge of specific selection (eg. protein):
```tcl
puts [vecsum [[atomselect top "protein"] get charge]]
```


   - This writes a crd file:
```tcl
[atomselect top all] writecrd MKK7_IB_l.crd
```



In [27]:
!vmd -dispdev text -e "$prot_name"_"$lig_name"_l.pgn

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 12 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 7003MB (42%)
Info) No CUDA accelerator devices available.
Info) Dynamically loaded 3 plugins in directory:
Info) /usr/local/lib/vmd/plugins/LINUXAMD64/molfile
ERROR) Illegal rendering mode: GLSL
after#0
after#1
/usr/local/lib/vmd/scripts/tcl8.5 /usr/local/lib/vmd/scripts /usr/local/lib/lib /Projects/johns/tcl/8.5.6/lib_LINUXAMD64/lib /usr/local/lib/vmd/scripts/vmd /usr/local/lib/vmd/plugins/LINUXAMD64/tcl /

## Generate .parm7 and .rst7 files

In [1]:
#strip non-xtal water within <cutoff> angstroms of ligand
#cutoff=5 is second water shell

def calcDistance(x1,y1,z1,x2,y2,z2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return dist

def stripwat(fn,prot_name,lig_name,cutoff):
    
    
    #determine xtal wat residues
    pdbfile="modeller/prot_wat.pdb"

    watres_l=['HOH','TIP3','WAT','TIP4']
    with open(pdbfile, 'r') as pdbfh :
        lines = pdbfh.readlines()
    watcount=0
    i=0
    prev_res=9999999
    for line in lines:
        res=line[22:26].strip()  
        if line[16:21].strip() in watres_l:

            if watcount==0:
                firstx=float(line[30:38].strip())
                firsty=float(line[38:46].strip())  
                firstz=float(line[46:54].strip())  
            if res != prev_res:
                watcount+=1
            #if lines[i+1][16:21].strip() not in watres_l:
            if 'O' in line[11:16].strip():
                lastx=float(line[30:38].strip())
                lasty=float(line[38:46].strip())  
                lastz=float(line[46:54].strip())  
        i+=1
        prev_res=line[22:26].strip()  
    dist1=np.round(calcDistance(firstx,firsty,firstz,lastx,lasty,lastz),1)
    print('Found {0} waters in {1}'.format(watcount,pdbfile))
    
    #change filenames
    !mv -nv "$fn".parm7 "$fn"_badwater.parm7
    !mv -nv "$fn".rst7 "$fn"_badwater.rst7
    reformat=!cpptraj -p "$fn"_badwater.parm7 -y "$fn".pdb -x "$fn"temp.pdb
    !mv "$fn"temp.pdb "$fn".pdb
    
    prev_res=999999
    maxwat=watcount
    pdbfile=fn+'.pdb'
    with open(pdbfile, 'r') as pdbfh :
        lines = pdbfh.readlines()
    for line in lines:
        res=line[22:26].strip()
        if line[16:21].strip() in watres_l:
            if maxwat==watcount:
                firstx=float(line[30:38].strip())
                firsty=float(line[38:46].strip())  
                firstz=float(line[46:54].strip())  
                firstres=line[22:26].strip()  
            if res != prev_res:
                watcount-=1
            if watcount==0 and 'O' in line[11:16].strip():
                lastx=float(line[30:38].strip())
                lasty=float(line[38:46].strip())  
                lastz=float(line[46:54].strip())  
                lastres=line[22:26].strip() 
                break
        prev_res=line[22:26].strip()  
    dist2=np.round(calcDistance(firstx,firsty,firstz,lastx,lasty,lastz),1)
                
               
            
    xtal_resrange=firstres+'-'+lastres
    print('Xtal water resrange is '+xtal_resrange)

    if dist1 != dist2: 
        print('ERROR: Distance between first and last waters in {0} ({1}) is different than'.format(pdbfile,dist1)) 
        print('\tthat in modeller/prot_wat.pdb ({0}). Exiting...'.format(dist2))
        sys.exit()
    
    
    #strip with cpptraj 
    strip_parm7_contents='''
    parm {0}_badwater.parm7
    reference {0}_badwater.rst7 1
    parmstrip (:{1}<:5)&:WAT&!(:{2})
    parmwrite out {0}.parm7
    go
    '''.format(fn,lig_name,xtal_resrange) 
    with open("strip_parm7.traj", "w") as cpptrajfh:
        cpptrajfh.write(strip_parm7_contents)
    trajlog_parm=!cpptraj -i strip_parm7.traj
    
    strip_rst7_contents='''
    parm {0}_badwater.parm7
    trajin {0}_badwater.rst7
    reference {0}_badwater.rst7 1
    autoimage
    strip (:{1}<:{2})&:WAT&!(:{3})
    trajout {0}.rst7
    go
    '''.format(fn,lig_name,cutoff,xtal_resrange) 
    with open("strip_rst7.traj", "w") as cpptrajfh:
        cpptrajfh.write(strip_rst7_contents)
    trajlog_rst=!cpptraj -i strip_rst7.traj
    
    #get new pdb and rename badwater files
    !mv -nv "$fn".pdb "$fn"_badwater.pdb
    !mv -nv "$fn".psf "$fn"_badwater.psf
    pdbtraj=!cpptraj -p "$fn".parm7 -y "$fn".rst7 -x "$fn".pdb
    print("Removing badwaters from pdb and making "+fn+".pdb")
    
    print(fn+'.parm7 and '+fn+'.rst7 are ready for minim, etc')
    
    return trajlog_parm,trajlog_rst

### a) Unlinked

In [29]:
#writes parmed script
u_fn = prot_name+"_"+lig_name+"_u"
u_parmed_contents = '''
chamber -psf {0}.psf -crd {0}.pdb \
-param toppar/par_all36m_prot.prm \
-param toppar/par_all36_cgenff.prm \
-str toppar/toppar_water_ions.str \
-param toppar/{2}.prm \
-param toppar/extra.prm \
nocondense -box {1}

#HMassRepartition

outparm {0}.parm7 {0}.rst7

#quit

'''.format(u_fn,pbc,lig_name)

with open(u_fn+".parmed", "w") as parmed_fh:
    parmed_fh.write(u_parmed_contents)
    
md('''
#### Run in the command line or next cell:
```
parmed -i {0}.parmed
```
#### Or copy/paste in parmed:
```python
chamber -psf {0}.psf -crd {0}.pdb -param toppar/par_all36m_prot.prm -param toppar/par_all36_cgenff.prm -str toppar/toppar_water_ions.str -param toppar/{2}.prm -param extra.prm nocondense -box {1}

#HMassRepartition

outparm {0}.parm7 {0}.rst7
```
'''.format(u_fn,pbc,lig_name))


#### Run in the command line or next cell:
```
parmed -i MKK7_IB_u.parmed
```
#### Or copy/paste in parmed:
```python
chamber -psf MKK7_IB_u.psf -crd MKK7_IB_u.pdb -param toppar/par_all36m_prot.prm -param toppar/par_all36_cgenff.prm -str toppar/toppar_water_ions.str -param toppar/IB.prm nocondense -box 93,93,93

#HMassRepartition

outparm MKK7_IB_u.parm7 MKK7_IB_u.rst7
```


In [30]:
!rm "$u_fn".parm7
!rm "$u_fn".rst7
!parmed -i "$u_fn".parmed


.     .       .  .   . .   .   . .    +  .
  .     .  :     .    .. :. .___---------___.
       .  .   .    .  :.:. _".^ .^ ^.  '.. :"-_. .
    .  :       .  .  .:../:            . .^  :.:\.
        .   . :: +. :.:/: .   .    .        . . .:\
 .  :    .     . _ :::/:               .  ^ .  . .:\
  .. . .   . - : :.:./.                        .  .:\
  .      .     . :..|:                    .  .  ^. .:|
    .       . : : ..||        .                . . !:|
  .     . . . ::. ::\(                           . :)/
 .   .     : . : .:.|. ######              .#######::|
  :.. .  :-  : .:  ::|.#######           ..########:|
 .  .  .  ..  .  .. :\ ########          :######## :/
  .        .+ :: : -.:\ ########       . ########.:/
    .  .+   . . . . :.:\. #######       #######..:/
      :: . . . . ::.:..:.\           .   .   ..:/
   .   .   .  .. :  -::::.\.       | |     . .:/
      .  :  .  .  .-:.":.::.\             ..:/ .~~~~~~~~~~~.
 .      -.   . . . .: .:::.:.\.           .:/,: Prepare 

In [31]:
#strip non-xtal water within <cutoff> angstroms of ligand
#cutoff=5 is second water shell
trajlog_parm,trajlog_rst=stripwat(u_fn,prot_name,lig_name,5)

Found 182 waters in xtal/xtal_prot_wat.pdb
renamed 'MKK7_IB_u.parm7' -> 'MKK7_IB_u_badwater.parm7'
renamed 'MKK7_IB_u.rst7' -> 'MKK7_IB_u_badwater.rst7'

CPPTRAJ: Trajectory Analysis. V5.1.0
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 05/04/22 01:06:07
| Available memory: 7.089 GB

	Reading 'MKK7_IB_u_badwater.parm7' as Amber Topology
	CHAMBER topology:
               >>>> CHARMM36 All-Hydrogen Parameter File for Proteins <<<<<<<<<<
	Radius Set: modified Bondi radii (mbondi)
	Reading 'MKK7_IB_u.pdb' as PDB
	Writing 'MKK7_IB_utemp.pdb' as PDB
---------- RUN BEGIN -------------------------------------------------

PARAMETER FILES (1 total):
 0: MKK7_IB_u_badwater.parm7, 75821 atoms, 24032 res, box: None, 23730 mol, 23589 solvent

INPUT TRAJECTORIES (1 total):
 0: 'MKK7_IB_u.pdb' is a PDB file, Parm MKK7_IB_u_badwater.parm7 (reading 1 of 1)
  Coordinate processing will occur on 1 frames.

OUTPUT TRAJECTORIES (1 total):
  'MKK7_IB_utemp.pdb' (1 frames

In [32]:
#uncomment log you want to check
trajlog_parm
#trajlog_rst

['',
 'CPPTRAJ: Trajectory Analysis. V5.1.0',
 '    ___  ___  ___  ___',
 '     | \\/ | \\/ | \\/ | ',
 '    _|_/\\_|_/\\_|_/\\_|_',
 '',
 '| Date/time: 05/04/22 01:06:08',
 '| Available memory: 7.072 GB',
 '',
 "INPUT: Reading input from 'strip_parm7.traj'",
 '  [parm MKK7_IB_u_badwater.parm7]',
 "\tReading 'MKK7_IB_u_badwater.parm7' as Amber Topology",
 '\tCHAMBER topology:',
 '               >>>> CHARMM36 All-Hydrogen Parameter File for Proteins <<<<<<<<<<',
 '\tRadius Set: modified Bondi radii (mbondi)',
 '  [reference MKK7_IB_u_badwater.rst7 1]',
 "\tReading 'MKK7_IB_u_badwater.rst7' as Amber Restart",
 "\tSetting active reference for distance-based masks: 'MKK7_IB_u_badwater.rst7'",
 '  [parmstrip (:IB<:5)&:WAT&!(:305-486)]',
 '\tStripping atoms in mask [(:IB<:5)&:WAT&!(:305-486)] (45) from MKK7_IB_u_badwater.parm7',
 '\tStripped parm: 75776 atoms, 24017 res, box: Cubic, 23715 mol, 23574 solvent',
 '  [parmwrite out MKK7_IB_u.parm7]',
 "\tWriting topology 0 (MKK7_IB_u_badwater.pa

### b) Linked

In [33]:
#writes parmed script
l_fn = prot_name+"_"+lig_name+"_l"
l_parmed_contents = '''
chamber -psf {0}.psf -crd {0}.pdb \
-param toppar/par_all36m_prot.prm \
-param toppar/par_all36_cgenff.prm \
-str toppar/toppar_water_ions.str \
-param toppar/{2}.prm \
-param toppar/{3}.prm \
-param toppar/extra.prm \
nocondense -box {1}

#HMassRepartition

outparm {0}.parm7 {0}.rst7

#quit

'''.format(l_fn,pbc,lig_name,patch_name)

with open(l_fn+".parmed", "w") as parmed_fh:
    parmed_fh.write(l_parmed_contents)
    
md('''
#### Run in the command line or next cell:
```
parmed -i {0}.parmed
```
#### Or copy/paste in parmed:
```python
chamber -psf {0}.psf -crd {0}.pdb -param toppar/par_all36m_prot.prm -param toppar/par_all36_cgenff.prm -str toppar/toppar_water_ions.str -param toppar/{2}.prm -param toppar/{3}.prm -param extra.prm nocondense -box {1}

#HMassRepartition

outparm {0}.parm7 {0}.rst7
```
'''.format(l_fn,pbc,lig_name,patch_name))


#### Run in the command line or next cell:
```
parmed -i MKK7_IB_l.parmed
```
#### Or copy/paste in parmed:
```python
chamber -psf MKK7_IB_l.psf -crd MKK7_IB_l.pdb -param toppar/par_all36m_prot.prm -param toppar/par_all36_cgenff.prm -str toppar/toppar_water_ions.str -param toppar/IB.prm -param toppar/LIGP.prm nocondense -box 93,93,93

#HMassRepartition

outparm MKK7_IB_l.parm7 MKK7_IB_l.rst7
```


In [34]:
!rm "$l_fn".parm7
!rm "$l_fn".rst7
!parmed -i "$l_fn".parmed


                                  /^^^/           /]
                                 /   ]           / ]
                         _______/    ]___       /  ]
                        /                \_____/   /
                      _/   [@]  \ \                \
                     /..         | |                ]
                      VVVvvv\    | |         _/\    ]
          P A R M E D       |               /    \  ]
                     AAA^^^^^              /       \]
                      \_________\   \_____/
                                \    \
                                 \____\

ParmEd: a Parameter file Editor


['MKK7_IB_l.parmed']
Reading actions from MKK7_IB_l.parmed

Creating chamber topology file from PSF MKK7_IB_l.psf, PAR files [toppar/par_all36m_prot.prm, toppar/par_all36_cgenff.prm, toppar/IB.prm, toppar/LIGP.prm], and STR files [toppar/toppar_water_ions.str]. Coords from MKK7_IB_l.pdb. Using CMAP. Box info [93.0, 93.0, 93.0, 90.0, 90.0, 90.0]. GB Radius se

In [35]:
#strip non-xtal water within <cutoff> angstroms of ligand
#cutoff=5 is second water shell
trajlog_parm,trajlog_rst=stripwat(l_fn,prot_name,lig_name,5)

Found 182 waters in xtal/xtal_prot_wat.pdb
renamed 'MKK7_IB_l.parm7' -> 'MKK7_IB_l_badwater.parm7'
renamed 'MKK7_IB_l.rst7' -> 'MKK7_IB_l_badwater.rst7'

CPPTRAJ: Trajectory Analysis. V5.1.0
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 05/04/22 01:10:40
| Available memory: 7.051 GB

	Reading 'MKK7_IB_l_badwater.parm7' as Amber Topology
	CHAMBER topology:
               >>>> CHARMM36 All-Hydrogen Parameter File for Proteins <<<<<<<<<<
	Radius Set: modified Bondi radii (mbondi)
	Reading 'MKK7_IB_l.pdb' as PDB
	Writing 'MKK7_IB_ltemp.pdb' as PDB
---------- RUN BEGIN -------------------------------------------------

PARAMETER FILES (1 total):
 0: MKK7_IB_l_badwater.parm7, 75821 atoms, 24032 res, box: None, 23729 mol, 23589 solvent

INPUT TRAJECTORIES (1 total):
 0: 'MKK7_IB_l.pdb' is a PDB file, Parm MKK7_IB_l_badwater.parm7 (reading 1 of 1)
  Coordinate processing will occur on 1 frames.

OUTPUT TRAJECTORIES (1 total):
  'MKK7_IB_ltemp.pdb' (1 frames

In [37]:
#uncomment log you want to check
trajlog_parm
#trajlog_rst

['',
 'CPPTRAJ: Trajectory Analysis. V5.1.0',
 '    ___  ___  ___  ___',
 '     | \\/ | \\/ | \\/ | ',
 '    _|_/\\_|_/\\_|_/\\_|_',
 '',
 '| Date/time: 05/04/22 01:10:40',
 '| Available memory: 7.026 GB',
 '',
 "INPUT: Reading input from 'strip_parm7.traj'",
 '  [parm MKK7_IB_l_badwater.parm7]',
 "\tReading 'MKK7_IB_l_badwater.parm7' as Amber Topology",
 '\tCHAMBER topology:',
 '               >>>> CHARMM36 All-Hydrogen Parameter File for Proteins <<<<<<<<<<',
 '\tRadius Set: modified Bondi radii (mbondi)',
 '  [reference MKK7_IB_l_badwater.rst7 1]',
 "\tReading 'MKK7_IB_l_badwater.rst7' as Amber Restart",
 "\tSetting active reference for distance-based masks: 'MKK7_IB_l_badwater.rst7'",
 '  [parmstrip (:IB<:5)&:WAT&!(:305-486)]',
 '\tStripping atoms in mask [(:IB<:5)&:WAT&!(:305-486)] (45) from MKK7_IB_l_badwater.parm7',
 '\tStripped parm: 75776 atoms, 24017 res, box: Cubic, 23714 mol, 23574 solvent',
 '  [parmwrite out MKK7_IB_l.parm7]',
 "\tWriting topology 0 (MKK7_IB_l_badwater.pa

#### Check parm7, rst7, and pdb

## QnA
- Any general checks you like to do?
    - Over first nanosecond, make sure bonds/dihedrals/etc are acting normal. eg doing eclipsed configs and are instead holding expected minima (gauche etc) 
    - QM investigation of dihedrals and bonds should be checked in theory (Gaussian). Evaluate energy of each rotation in a dihedral. Get stuff running first and come to this if there is any cause for concern.
    - Check penalties. Could give wrong phase. If penalty is over cov bond, investigate more. Is it expected? Minima are in right locations? Good analogy if I have unsual atom environment?
    - Check protonation states before charmmgui (protein) and paramchem (ligand). 
    - Check stereochemistry is right whenever you regenerate molecule through something (eg paramchem).
    - Check that charge is integer
    - Check that ring params are symmetrical and atomtypes are right
    - Check ligand and protein clashes 
   
- Is the charge-checking process right? Am I missing anything?
    - Maintain symmetry when fudging. Dont make pos to neg and vice versa. 
    - Add ions if integer charge changes from patch. Can add with text editing. 
    
- Will waters in the pocket be replaced by psfgen when combining the ligand and solvated protein? 
    - Waters that overlap can be fixed with visual inspection. They will usually be removed from the pocket durign equil. If system explodes, check. 
    
- Is it fine to have the linked ligand be on a different chain? 
    - Probably. Chain isnt used in MD. 

- Do I fudge to get only ligand atoms to 0 total charge or do I include the difference in charge of patched side chain atoms?
    - Result based.
    - Can't go past 4 since cgenff doesnt go past 3 atoms. 
    - Avoid fudging anything on alpha carbon. 
    - Change beta carbon 
    
- If aspartic acids are separated from solvent, local pH causes it to be protonated. Is this a pre-charmmgui/resname fix?
    - Pre-charmmgui. Text edit PDB for protein. 
    
- Psfgen doesnt put XPLOR in header even when writing as XPLOR format. Is this ok?
    - Yes but errors will make it clear. 
    
- Does my ligp.prm need anything added/removed?

- When (not) to use nocondense in chamber?
    - Removes duplicates dihedrals (check for details)
    - 

### TODO
- decrease amount of dependencies 
- automate some setup steps as an option
- improve "set general variable" section
- add error detection and more print lines in general
- does step3 pdb have to be tip3? (reres wat section)
- automatically load checks into vmd 
- what does this mean in setup steps:
    - #manually change desired atom name and cut and paste the line to the desired spot. 
    - #eg. change added hydrogen from H26 (last one) to H00 (now first)